In [11]:
# modules
import os
import pandas as pd
import numpy as np
from tqdm import  tqdm
from os.path import basename as bn, join, split as sp

from sklearn.utils import shuffle

# DATASET Path and Constants

In [12]:
# Read paths
ROOT_TIMIT_DATA_PATH = "/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT"

# Write paths
ALL_EXP_FOLDER = "./exports/"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(ALL_EXP_FOLDER) #make export folder

# Vowel info Export CSV filename
ALL_TIMIT_VOWELS_IMP_FILENAME = "a_all-timit_vowels.csv"
ALL_TIMIT_VOWELS_IMP_FILEPATH = join(ALL_EXP_FOLDER, ALL_TIMIT_VOWELS_IMP_FILENAME)

# Vowel subset Export CSV filename
SUBSET_TIMIT_VOWELS_EXP_FILENAME = "b_subset-timit_vowels_vowlimLIM.csv"
SUBSET_TIMIT_VOWELS_EXP_FILEPATH = join(ALL_EXP_FOLDER, SUBSET_TIMIT_VOWELS_EXP_FILENAME)

# TIMIT SAMPLING RATE
TIMIT_AUDIO_FS = 16000

### Import All TIMIT Vowel Info dataframe

In [13]:
ALL_TIMIT_VOWELS_DF = pd.read_csv(ALL_TIMIT_VOWELS_IMP_FILEPATH)
ALL_TIMIT_VOWEL_LIST = pd.unique(ALL_TIMIT_VOWELS_DF["vowel_name"])
ALL_TIMIT_VOWELS_DF["vowel_name"].value_counts()

ix      11587
iy       9663
ih       6760
ae       5404
eh       5293
ax       4956
axr      4790
aa       4197
ao       4096
ay       3242
ah       3185
ey       3088
ow       2913
er       2846
ux       2488
oy        947
aw        945
uh        756
uw        725
ax-h      493
Name: vowel_name, dtype: int64

## FUNCTION: Make Vowel info Subset

In [14]:
def make_vowel_subset(vowel: str, df: pd.DataFrame=ALL_TIMIT_VOWELS_DF,  limit=2) -> pd.DataFrame:
    filt = df["vowel_name"] == vowel
    vowel_df = df[filt]

    filt_sex = lambda s: vowel_df["sex"] == s
    male_vowel_df = shuffle(vowel_df[filt_sex("M")], random_state=2)
    female_vowel_df = shuffle(vowel_df[filt_sex("F")], random_state=4)

    m_size = limit if len(male_vowel_df) > limit else len(male_vowel_df)
    f_size = limit if len(female_vowel_df) > limit else len(female_vowel_df)

    male_vowel_df   = male_vowel_df[:m_size]
    female_vowel_df = female_vowel_df[:f_size]

    vowel_df = pd.concat([male_vowel_df, female_vowel_df]) 

    return vowel_df
    
make_vowel_subset(
    vowel=ALL_TIMIT_VOWEL_LIST[np.random.randint(0, len(ALL_TIMIT_VOWEL_LIST))],
    df=ALL_TIMIT_VOWELS_DF,
    limit=5
)

,index,audio_filepath,wav_file,person_id,sex,start_sample,end_sample,duration_sample,start_second,end_second,duration_second,vowel_name
52831,5,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MTRT0,M,19040,20720,1680,1.190000,1.295000,0.105000,iy
23733,7,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA1.WAV,MMGG0,M,31768,33269,1501,1.985500,2.079312,0.093812,iy
32508,8,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX62.WAV,MPPC0,M,27195,28365,1170,1.699688,1.772812,0.073125,iy
31267,2,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MKAH0,M,9800,10778,978,0.612500,0.673625,0.061125,iy
60006,14,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI1421.WAV,MRML0,M,79791,82630,2839,4.986937,5.164375,0.177437,iy
2205,9,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI730.WAV,FRAM1,F,34066,35184,1118,2.129125,2.199000,0.069875,iy
53370,8,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI567.WAV,FDMY0,F,32279,33690,1411,2.017438,2.105625,0.088188,iy
62908,5,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI2329.WAV,FSBK0,F,20810,22680,1870,1.300625,1.417500,0.116875,iy
44874,7,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA1.WAV,FDKN0,F,33370,35181,1811,2.085625,2.198812,0.113187,iy
27818,2,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI805.WAV,FRLL0,F,7520,9040,1520,0.470000,0.565000,0.095000,iy


### Create and export Vowel Info SUBSET dataframe

In [15]:
VOWEL_LIMIT = 2
SUBSET_TIMIT_VOWELS_DF = pd.concat([make_vowel_subset(vowel, limit=VOWEL_LIMIT) for vowel in tqdm(ALL_TIMIT_VOWEL_LIST)])

SUBSET_TIMIT_VOWELS_DF.reset_index(inplace=True)
SUBSET_TIMIT_VOWELS_DF.index.name = "index"
SUBSET_TIMIT_VOWELS_DF.drop(columns=["index", "level_0"], inplace=True)

SUBSET_TIMIT_VOWELS_DF.to_csv(SUBSET_TIMIT_VOWELS_EXP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}"), index=True)

SUBSET_TIMIT_VOWELS_DF 

100%|██████████| 20/20 [00:00<00:00, 85.04it/s]


,audio_filepath,wav_file,person_id,sex,start_sample,end_sample,duration_sample,start_second,end_second,duration_second,vowel_name
index,,,,,,,,,,,
0,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MTRT0,M,19040,20720,1680,1.190000,1.295000,0.105000,iy
1,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA1.WAV,MMGG0,M,31768,33269,1501,1.985500,2.079312,0.093812,iy
2,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI730.WAV,FRAM1,F,34066,35184,1118,2.129125,2.199000,0.069875,iy
3,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI567.WAV,FDMY0,F,32279,33690,1411,2.017438,2.105625,0.088188,iy
4,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MKLS1,M,28699,31624,2925,1.793688,1.976500,0.182812,ae
...,...,...,...,...,...,...,...,...,...,...,...
75,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX338.WAV,FECD0,F,17880,18920,1040,1.117500,1.182500,0.065000,uh
76,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX306.WAV,MWVW0,M,2912,3444,532,0.182000,0.215250,0.033250,ax-h
77,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX401.WAV,MJDA0,M,21000,21377,377,1.312500,1.336062,0.023563,ax-h
